In [2]:
import pymongo
connection = pymongo.MongoClient('localhost', 27017)
db = connection.test

In [5]:
connection = pymongo.MongoClient('localhost', 27017)

In [6]:
db = connection.test

In [7]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test')

# 3.1 루비를 통해 보는 MongoDB

모든 드라이버는 직관적이고 해당 프로그래밍 언어에 적합한 API로 설계되었으므로 많은 애플리케이션에서 MongoDB에 대한 유일한 인터페이스로 드라이버만 사용할 수도 있다. 
<br>
루비에서는 Bson을 설치하는게 더 효율적인 핸들링을 제공해준다. 

In [9]:
# 모든 몽고디비 드라이버는 해당 프로그래밍 언어에서 가장 자연스러운 방법으로 도큐먼트를 표현하도록 설계되었다. 
# 루비에서는 : 대신에 =>을 사용한다. 
# 따라서 루비에서는 해시 데이터 구조가 기징 힙리적이다. 
smith = {'last_name' : 'smith', 'age' : 30}
jones = {'last_name' : 'jones', 'age' : 40}

In [10]:
db.users.insert(smith)
db.users.insert(jones)

/Users/gunwook/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  """Entry point for launching an IPython kernel.


ObjectId('5ec69dfb6775247830b23f86')

## 3.1.3 쿼리와 커서
루비 드라이버에는 데이터에 접근하고 대부분의 상세한 사항들을 다루는 풍부한 인터페이스가 정의되었다. 

In [26]:
print(db.users.find_one({'last_name':'smith'}))
# gt는 greater than의 약자다. 
print(db.users.find_one({'age': {'$gt' : 30}}))

{'_id': ObjectId('5ec69dfb6775247830b23f86'), 'last_name': 'smith', 'age': 30}
{'_id': ObjectId('5ec69e026775247830b23f87'), 'last_name': 'jones', 'age': 40}


## 3.1.4 업데이트와 삭제
업데이트는 최소한 두 개의 매개변수, 즉 쿼리 셀렉터와 엡데이트 도큐먼트가 필요하다. 

In [31]:
# $set은 업데이트 시 사용한다. 
db.users.update_one({'last_name':'smith'}, {'$set':{'city':'Chicago'}})

In [35]:
# 업데이트가 된 것을 확인할 수 있다. 
db.users.find_one()

{'_id': ObjectId('5ec69dfb6775247830b23f86'),
 'last_name': 'smith',
 'age': 30,
 'city': 'Chicago'}

In [34]:
# 업데이트할 스미스가 여러명이면 update_many를 사용하자
db.users.update_many({'last_name':'smith'}, {'$set':{'city':'Chicago'}})

In [36]:
# 데이터 삭제는 remove 메서드를 사용하자.
db.users.delete_one({'age': {'$gt' : 30}})

In [37]:
# 여러개를 없앨 때는 delete_many()
db.users.delete_many({'age': {'$gt' : 30}})

In [38]:
# 모든 내용을 삭제해준다.
db.users.drop()

## 3.1.5 데이터베이스 명령어
listDatabases 명령을 통해 드라이버에서 어떻게 데이터베이스 명령어를 실행할 수 있는지 확인할 수 있다.

# 3.2 드라이버 작동 원리
모든 몽고디비 드라이버는 세 가지 주요한 기능을 수행한다. 
1. 모든 도큐먼트의 _id 필드에 디폴트로 저장되는 값인 몽고디비 객체 ID를 생성한다. 
2. 드라이버는 특정 언어로 표현된 도큐먼트와 몽고디비의 이진 데이터 포맷인 BSON 사이의 변환을 수행한다. 
3. 드라이버는 몽고디비의 와이어 프로토콜을 사용해서 네트워크 소켓을 통해 데이터베이스와 통신한다. 

## 3.2.1 객체 ID 생성
모든 몽고디비 도큐먼트는 프라이머리 키가 필요하다. 개발자가 _id 필드에 임의의 값을 넣는 것도 가능하지만, 주어지지 않을 경우 몽고디비의 객체 ID가 사용된다. 코어 서버에서 도큐먼트를 전송하기 전에 드라이버는 _id가 필요한지 확인 후 필드가 없으면 객체 ID를 생성하게 된다. 
<br>
몽고디비의 객체 ID는 전역적으로 고유하도록 설계되어 있고, 이는 특정 맥락에서 고유하도록 보장함을 의미한다. 객체 ID는 12바이트의 16진수로 표현된다.
- 4바이트: 유닉스의 타임스탬프
- 3바이트: 서버의 id
- 2바이트: 포로세스 id
- 3바이트: 객체 id가 생성될 때마다 하나씩 증가하는 프로세스의 로컬 카운터. 
<br>
몽고디비 객체 ID를 사용할 때 장점은 객체 ID가 타임스탬프를 포함하고 있다는 것이다. 따라서 도큐먼트의 생성시간을 알 수 있다. 